In [96]:
from datetime import date, datetime, timezone

import pytz
import pandas as pd
import numpy as np
import os
import re

# Extract Data

In [97]:
data_recruitment_path = "/content/drive/MyDrive/Purwadhika/data_sources/data_reqruitment"

files = os.listdir(data_recruitment_path)
print(files[:5])

['data_requirements.csv']


In [98]:
df = pd.read_csv(data_recruitment_path + '/' + files[0])
df = df.drop('Unnamed: 0', axis=1)

df.head(2)

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates
0,PCS Global Tech\n4.7,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,2024-06-12 00:00:00+07:00


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           1551 non-null   object 
 1   company_rating    1358 non-null   float64
 2   location          1554 non-null   object 
 3   job_title         1554 non-null   object 
 4   job_description   1554 non-null   object 
 5   salary_estimate   1277 non-null   object 
 6   company_size      1442 non-null   object 
 7   company_type      1442 non-null   object 
 8   company_sector    1260 non-null   object 
 9   company_industry  1260 non-null   object 
 10  company_founded   1131 non-null   float64
 11  company_revenue   1442 non-null   object 
 12  dates             1555 non-null   object 
dtypes: float64(2), object(11)
memory usage: 158.1+ KB


# Data Transformation & Demography per column

## company

In [100]:
display(df['company'].describe())
print(f"null value count: {df['company'].isnull().sum()}")

,company
count,1551
unique,1236
top,Apple\n4.2
freq,14


null value count: 4


In [101]:
df[df['company'].isna()]

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates
323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-16 00:00:00+07:00
525,NaN,NaN,"Phoenix, AZ",Data Warehouse Engineer - Remote,"At UnitedHealthcare, we’re simplifying the hea...","$100,450 /yr (est.)",10000+ Employees,Company - Public,Healthcare,Health Care Services & Hospitals,1977.0,$10+ billion (USD),2024-06-18 00:00:00+10:00
529,NaN,NaN,"Minnetonka, MN",Data Engineer - Remote,"At UnitedHealthcare, we’re simplifying the hea...","$100,450 /yr (est.)",10000+ Employees,Company - Public,Healthcare,Health Care Services & Hospitals,1977.0,$10+ billion (USD),2024-06-18 00:00:00-04:00
667,NaN,NaN,"Englewood, CO",Senior Data Engineer,Department Summary\n\nDISH is a Fortune 200 co...,"$115,625 /yr (est.)",10000+ Employees,Company - Public,Telecommunications,"Cable, Internet & Telephone Providers",1980.0,$10+ billion (USD),2024-06-20 00:00:00-05:00


In [102]:
df = df.drop(323)

In [103]:
len(df[df['company'].str.contains('\n', na=False)])

1358

In [104]:
# only take the value before the first \n
df['company'] = df['company'].str.split('\n').str[0]

## company rating

In [105]:
display(df['company_rating'].describe())
print(f"null value count: {df['company_rating'].isnull().sum()}")

,company_rating
count,1358.000000
mean,3.925626
std,0.535826
min,1.000000
25%,3.600000
50%,3.900000
75%,4.200000
max,5.000000


null value count: 196


## location

In [106]:
display(df['location'].describe())
print(f"null value count: {df['location'].isnull().sum()}")
print(f"Unique locations: {df['location'].nunique()}")
print(df['location'].value_counts().head(10))

,location
count,1554
unique,419
top,Remote
freq,234


null value count: 0
Unique locations: 419
location
Remote               234
United States         58
Washington, DC        43
New York, NY          40
Atlanta, GA           39
Dallas, TX            37
Austin, TX            35
Seattle, WA           29
San Francisco, CA     28
Boston, MA            25
Name: count, dtype: int64


## job title

In [107]:
display(df['job_title'].describe())
print(f"null value count: {df['job_title'].isnull().sum()}")
print(f"Unique job_titles: {df['job_title'].nunique()}")
print(df['job_title'].value_counts())

,job_title
count,1554
unique,781
top,Data Engineer
freq,356


null value count: 0
Unique job_titles: 781
job_title
Data Engineer                                                   356
Senior Data Engineer                                            132
Sr. Data Engineer                                                36
Azure Data Engineer                                              22
AWS Data Engineer                                                17
                                                               ... 
Data Engineer/Analyst / W2 /USC or GC (GC EAD) or H4 holders      1
Network Engineer/Data Center                                      1
Azure Data Factory Engineer                                       1
Data Engineer III – Hybrid                                        1
Cloud Data Engineer - Hybrid - Roseville, CA                      1
Name: count, Length: 781, dtype: int64


In [108]:
def get_role(job_title, role, keywords):
  if any(keyword in job_title.lower() for keyword in keywords):
    return role

df['role'] = df['job_title'].apply(get_role, args=('data engineer', ['data engineer', 'analytics engineer']))

In [109]:
df[df['role'] != 'data engineer']['job_title'].value_counts()

,count
job_title,
Data Center Engineer,8
Data Operations Engineer,6
Data Integration Engineer,5
Data Center Network Engineer,4
Data Visualization Engineer,3
...,...
Data Center Lead Engineer,1
"AIML - Sr Machine Learning Engineer, Data & ML Innovation",1
Product Data Management Engineer,1


In [110]:
df['role'] = 'data engineer'

In [111]:
keywords = {
    'Internship': ['intern', 'internship'],
    'Junior': ['junior', 'jr'],
    'Senior': ['senior', 'sr'],
    'Manager': ['manager', 'lead', 'head'],
}

def get_job_level(title):
    title = title.lower()
    for category, keyword_list in keywords.items():
        if any(keyword in title for keyword in keyword_list):
            return category
    return np.nan

In [112]:
df['level'] = df['job_title'].apply(get_job_level)

In [113]:
df['level'].value_counts()

,count
level,
Senior,393
Manager,60
Junior,19


## salary estimate

In [114]:
def process_salary(salary):
  if pd.isnull(salary):
    return pd.Series([None, None])
  value = re.findall(r"[\d,.]+", salary)
  value = float(value[0].replace(",", "")) if value else None
  period = re.findall(r"(hr|yr|month|week|day)", salary.lower())
  period = period[0] if period else None
  return pd.Series([value, period])

# Apply the function to create new columns
df[['salary_value', 'salary_period']] = df['salary_estimate'].apply(process_salary)

# Display the processed columns
df.head()

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates,role,level,salary_value,salary_period
0,PCS Global Tech,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00,data engineer,NaN,70000.0,yr
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,2024-06-12 00:00:00+07:00,data engineer,NaN,42.5,hr
2,Clairvoyant,4.4,Remote,Data Engineer (MDM),Required Skills:\nMust have 5-8+ Years of expe...,$67.50 /hr (est.),51 to 200 Employees,Company - Private,Pharmaceutical & Biotechnology,Biotech & Pharmaceuticals,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00,data engineer,NaN,67.5,hr
3,Apple,4.2,"Cupertino, CA",Data Engineer,"Summary\nPosted: Dec 22, 2021\nWeekly Hours: 4...",NaN,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,1976.0,$10+ billion (USD),2024-06-12 00:00:00-05:00,data engineer,NaN,NaN,None
4,Skytech Consultancy Services,5.0,"Baltimore, MD",Data Engineer,Description of Work:\nTechnical experience in ...,$65.00 /hr (est.),1 to 50 Employees,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-04:00,data engineer,NaN,65.0,hr


## company size

In [115]:
display(df['company_size'].describe())
print(f"null value count: {df['company_size'].isnull().sum()}")
print(f"Unique company_sizes: {df['company_size'].nunique()}")
print(df['company_size'].value_counts())

,company_size
count,1442
unique,8
top,10000+ Employees
freq,436


null value count: 112
Unique company_sizes: 8
company_size
10000+ Employees           436
51 to 200 Employees        221
1001 to 5000 Employees     203
1 to 50 Employees          199
201 to 500 Employees       138
Unknown                     93
501 to 1000 Employees       85
5001 to 10000 Employees     67
Name: count, dtype: int64


## company type

In [116]:
display(df['company_type'].describe())
print(f"null value count: {df['company_type'].isnull().sum()}")
print(f"Unique company_types: {df['company_type'].nunique()}")
print(df['company_type'].value_counts())

,company_type
count,1442
unique,12
top,Company - Private
freq,726


null value count: 112
Unique company_types: 12
company_type
Company - Private                 726
Company - Public                  565
Nonprofit Organization             46
Subsidiary or Business Segment     32
Government                         15
College / University               15
Contract                           12
Unknown                            12
Private Practice / Firm            10
Hospital                            5
Self-employed                       3
School / School District            1
Name: count, dtype: int64


## company sector

In [117]:
display(df['company_sector'].describe())
print(f"null value count: {df['company_sector'].isnull().sum()}")
print(f"Unique company_sectors: {df['company_sector'].nunique()}")
print(df['company_sector'].value_counts())

,company_sector
count,1260
unique,25
top,Information Technology
freq,507


null value count: 294
Unique company_sectors: 25
company_sector
Information Technology                         507
Financial Services                             118
Healthcare                                      85
Management & Consulting                         73
Manufacturing                                   73
Insurance                                       47
Aerospace & Defense                             44
Retail & Wholesale                              37
Media & Communication                           33
Energy, Mining & Utilities                      33
Human Resources & Staffing                      31
Education                                       28
Pharmaceutical & Biotechnology                  28
Government & Public Administration              25
Construction, Repair & Maintenance Services     16
Transportation & Logistics                      14
Arts, Entertainment & Recreation                13
Restaurants & Food Service                      11
Real Estate       

## company industry

In [118]:
display(df['company_industry'].describe())
print(f"null value count: {df['company_industry'].isnull().sum()}")
print(f"Unique company_industrys: {df['company_industry'].nunique()}")
print(df['company_industry'].value_counts())

,company_industry
count,1260
unique,83
top,Information Technology Support Services
freq,215


null value count: 294
Unique company_industrys: 83
company_industry
Information Technology Support Services     215
Enterprise Software & Network Solutions     105
Computer Hardware Development                81
Health Care Services & Hospitals             80
Internet & Web Services                      63
                                           ... 
Convenience Stores                            1
Medical Testing & Clinical Laboratories       1
Laundry & Dry Cleaning                        1
Consumer Electronics & Appliances Stores      1
Consumer Product Rental                       1
Name: count, Length: 83, dtype: int64


## company founded

In [119]:
display(df['company_founded'].describe())
print(f"null value count: {df['company_founded'].isnull().sum()}")
print(f"Unique company_foundeds: {df['company_founded'].nunique()}")
print(df['company_founded'].value_counts())

,company_founded
count,1131.000000
mean,1975.934571
std,51.461792
min,1636.000000
25%,1965.000000
50%,1997.000000
75%,2009.000000
max,2022.000000


null value count: 423
Unique company_foundeds: 168
company_founded
2004.0    38
2012.0    34
1994.0    32
2015.0    32
2017.0    32
          ..
1863.0     1
1910.0     1
1859.0     1
1918.0     1
1981.0     1
Name: count, Length: 168, dtype: int64


In [120]:
df['company_age'] = 2024 - df['company_founded']

## company revenue

In [121]:
display(df['company_revenue'].describe())
print(f"null value count: {df['company_revenue'].isnull().sum()}")
print(f"Unique company_revenues: {df['company_revenue'].nunique()}")
print(df['company_revenue'].value_counts())

,company_revenue
count,1442
unique,10
top,Unknown / Non-Applicable
freq,551


null value count: 112
Unique company_revenues: 10
company_revenue
Unknown / Non-Applicable            551
$10+ billion (USD)                  288
$1 to $5 billion (USD)              126
$100 to $500 million (USD)          110
$5 to $25 million (USD)             102
$25 to $100 million (USD)            96
$5 to $10 billion (USD)              54
$1 to $5 million (USD)               46
$500 million to $1 billion (USD)     43
Less than $1 million (USD)           26
Name: count, dtype: int64


## dates

In [122]:
df['dates']

,dates
0,2024-06-12 00:00:00-10:00
1,2024-06-12 00:00:00+07:00
2,2024-06-12 00:00:00-10:00
3,2024-06-12 00:00:00-05:00
4,2024-06-12 00:00:00-04:00
...,...
1550,2024-07-01 00:00:00+08:00
1551,2024-07-01 00:00:00-05:00
1552,2024-07-01 00:00:00+01:00
1553,2024-07-01 00:00:00-10:00


In [127]:
def convert_to_jkt(datetime_str):
    local_dt = pd.to_datetime(datetime_str)
    utc_dt = local_dt.tz_convert('UTC')
    jkt_dt = utc_dt.tz_convert('Asia/Bangkok')
    return jkt_dt

In [129]:
# Convert the datetime column to datetime objects with timezone awareness
df['jkt_date'] = df['dates'].apply(convert_to_jkt)

# Display the DataFrame
df.head(2)

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates,role,level,salary_value,salary_period,company_age,jkt_date
0,PCS Global Tech,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00,data engineer,NaN,70000.0,yr,NaN,2024-06-12 17:00:00+07:00
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,2024-06-12 00:00:00+07:00,data engineer,NaN,42.5,hr,NaN,2024-06-12 00:00:00+07:00


In [132]:
today = pd.Timestamp(datetime.now(pytz.timezone('Asia/Jakarta')))

df['days_since_posted'] = (today - df['jkt_date']).dt.days
df['days_since_posted'].describe()

,days_since_posted
count,1554.000000
mean,100.563063
std,5.656759
min,90.000000
25%,96.000000
50%,101.000000
75%,105.750000
max,110.000000
